# Run all adverSCarial attacks of the publication:
- on all clusters on the test dataset from pbmc3k
- single-gene attacks
- max-changes attacks
- perc1, perc99, decile+5, negative-aberrant, positive-aberrant modifications
- CGD attacks with several alpha and epsilon parameters
- possibility to run mono or multi thread

In [1]:
lapply(c("dplyr","Seurat","HGNChelper", "adverSCarial", "IRdisplay"), library, character.only = T)
options(warn=-1)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

‘SeuratObject’ was built under R 4.3.0 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
 [1] "Seurat"       "SeuratObject" "sp"           "dplyr"        "stats"       
 [6] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[11] "base"        

[[3]]
 [1] "HGNChelper"   "Seurat"       "SeuratObject" "sp"           "dplyr"       
 [6] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[11] "methods"      "base"        

[[4]]
 [1] "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject" "sp"          
 [6] "dplyr"        "stats"        "graphics"     "grDevices"    "utils"       
[11] "datasets"     "methods"      "base"        

[[5]]
 [1] "IRdisplay"    "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject"
 [6] "sp"           "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"

In [3]:
bd <- "repr_data/"
source("load_classifiers.r")

# Load test data as:
- single cell experiment object
- data.frame
- seurat object

In [4]:
so_pbmc_test <- readRDS(paste0(bd, "v2//pbmc_test.rds"))
clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
names(clusters_so) <- rownames(so_pbmc_test@meta.data)

In [5]:
sce_pbmc_test <- readRDS(paste0(bd, "v2//sce_pbmc_test.rds"))
clusters_sce = so_pbmc_test@meta.data$chr_seurat_cluster
names(clusters_sce) <- rownames(so_pbmc_test@meta.data)

In [6]:
df_pbmc_test <- read.table(paste0(bd, "v2//seurat_scaled_pbmc_test.txt"))
expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
clusters_df <- df_pbmc_test$y
names(clusters_df) <- rownames(df_pbmc_test)

In [10]:
mySignGenes <- getSignGenes(expr_df, clusters_df, verbose=T)
head(mySignGenes)

,gene,pval
,<chr>,<dbl>
HLA.DRA,HLA.DRA,1.004947e-70
PRF1,PRF1,6.443834e-62
NKG7,NKG7,1.070165e-79
FCER1A,FCER1A,2.765930e-18
TYROBP,TYROBP,8.127634e-85
IL32,IL32,1.260937e-66


In [7]:
# Function to export results from the CGD attacks
exportResults <- function(acbc, baseDir, expID, strClassifier){
    write.table(acbc$byGeneSummary,
        paste0(baseDir,"/byGeneSummary_",expID,".txt"), row.names=F)
    display(acbc$oneRowSummary)
    dfOneRowSummary <- as.data.frame(matrix(nrow=0, ncol=17))
    dfOneRowSummary <- rbind(dfOneRowSummary, c(strClassifier, acbc$oneRowSummary))
    colnames(dfOneRowSummary) <- c("classifier", "cluster","cluster_size", "targetCluster",
                                  "alpha","epsilon", "changeClust",
                                  "nbModifiedGenes", "nbTestedGenes",
                                  "oriPourcStart", "targPourcStart",
                                  "oriPourcEnd", "targPourcEnd",
                                   "wholeClust_oriOdd_beg", "wholeClust_targOdd_beg",
                                  "wholeClust_oriOdd_end", "wholeClust_targOdd_end")
    
    
    write.table(dfOneRowSummary,
                paste0(baseDir,"oneRowSummary_",expID,".txt"),
                row.names=F)
}

In [8]:
# Wrap every possible attacks in one function
# Can handle a list on attacks to run
# Possibility to multi-threading
runAttacks <- function(attackList, monoOrMulti="multi", nbThreads=10){
    for (attack in attackList){
        aSplit <- unlist(strsplit(attack, "_"))
        strClassifier <- aSplit[1]
        display(strClassifier)
        myAttack <- aSplit[2]
        myClassifier <- classifList[[strClassifier]]$fct
        inputType <- classifList[[strClassifier]]$inputType
        display(inputType)
        mySlot <- classifList[[strClassifier]]$slot
        expr <- ""
        # Set the expr and the clusters variables from the valid source for the classifier
        if(inputType == "data.frame"){
            expr <- expr_df
            clusters <- clusters_df
        }
        if(tolower(inputType) == "seurat"){
            expr <- so_pbmc_test
            clusters <- clusters_so
        }
        if (tolower(inputType) == "singlecellexperiment"){
            expr <- sce_pbmc_test
            clusters <- clusters_sce
        }
        # Single-gene attack
        if(myAttack == "singleGene"){
            modif <- aSplit[3]
            if(monoOrMulti == "mono"){
                for( myClust in unique(clusters)[9:1]){
                    filePath <- paste0(bd,"v2/results/",strClassifier,"_",myAttack,"_",modif,"__",myClust,"__.txt")
                    message(filePath)
                    # Run only if file does not exist yet
                    if (!file.exists(filePath)){
                        if (modif == "positiveAberrant"){
                            modif = "positive_aberrant"
                        }
                        if (modif == "negativeAberrant"){
                            modif = "negative_aberrant"
                        }
                        # Run single-gene attack
                        min_temp = advSingleGene(expr,
                                                 clusters,
                                                 myClust,
                                                 myClassifier, verbose=T,
                                                 argForClassif=inputType,
                                                 argForModif=inputType,
                                                 advMethod=modif,
                                                 slot=mySlot)
                        df_4export = data.frame(genes = names(min_temp@values),
                               cell_type=unname(unlist(lapply(min_temp@values, function(x)x[1]))))
                        # Export results
                        write.table(df_4export, filePath)
                    }
                }
            } else {
                # Prepare for the multi-threading
                library(parallel)
                cl <- makeCluster(nbThreads)
                all_vars <- ls(globalenv())
                clusterExport(cl, varlist = all_vars)
                message(paste0(strClassifier,"_",myAttack,"_",modif))
                parLapply(cl, unique(clusters), function(myCluster) {
                    library(adverSCarial)
                    library(scAnnotatR)
                    library(stringr)
                    library(dplyr)
                    library(Seurat)
                    library(HGNChelper)
                    filePath <- paste0(bd,"v2/results/",strClassifier,"_",myAttack,"_",modif,"__",myClust,"__.txt")
                    message(filePath)
                    # Run only if file does not exist yet
                    if (!file.exists(filePath)){
                        if (modif == "positiveAberrant"){
                            modif = "positive_aberrant"
                        }
                        if (modif == "negativeAberrant"){
                            modif = "negative_aberrant"
                        }
                        # Run single-gene attack
                        min_temp = advSingleGene(expr,
                                                 clusters,
                                                  myClust,
                                                 myClassifier, verbose=T,
                                                 argForClassif=inputType,
                                                 argForModif=inputType,
                                                 advMethod=modif,
                                                 slot=mySlot)
                        df_4export = data.frame(genes = names(min_temp@values),
                               cell_type=unname(unlist(lapply(min_temp@values, function(x)x[1]))))
                        # Export results
                        write.table(df_4export, filePath)
                    }
                })
            }
        }
        # Max-change attack
        if(myAttack == "maxChange"){
            modif <- aSplit[3]
            if(monoOrMulti == "mono"){
                for( myClust in unique(clusters)){
                    filePath <- paste0(bd,"/v2/results/",strClassifier,"_",myAttack,"_",modif,"__",myClust,"__maxSplitSize=20.txt")
                    message(filePath)
                    # Run only if file does not exist yet
                    if (!file.exists(filePath)){
                        if (modif == "positiveAberrant"){
                            modif = "positive_aberrant"
                        }
                        if (modif == "negativeAberrant"){
                            modif = "negative_aberrant"
                        }
                        max_temp = advMaxChange(expr,
                                    clusters,
                                    myClust,
                                    myClassifier, verbose=T,
                                    argForClassif=inputType,
                                    argForModif=inputType,
                                    advMethod=modif,
                                    slot=mySlot,
                                    maxSplitSize=20)
                        # Export results
                        write.table(max_temp@values, filePath)
                    }
                }
            } else {
                # Prepare for the multi-threading
                library(parallel)
                cl <- makeCluster(nbThreads)
                all_vars <- ls(globalenv())
                clusterExport(cl, varlist = all_vars)
                message(paste0(strClassifier,"_",myAttack,"_",modif))
                parLapply(cl, unique(clusters), function(myCluster) {
                    library(adverSCarial)
                    library(scAnnotatR)
                    library(stringr)
                    library(dplyr)
                    library(Seurat)
                    library(HGNChelper)
                    filePath <- paste0(bd, "/v2/results/",strClassifier,"_",myAttack,"_",modif,"__",myClust,"__.txt")
                    message(filePath)
                    # Run only if file does not exist yet
                    if (!file.exists(filePath)){
                        if (modif == "positiveAberrant"){
                            modif = "positive_aberrant"
                        }
                        if (modif == "negativeAberrant"){
                            modif = "negative_aberrant"
                        }
                        max_temp = advMaxChange(expr,
                                    clusters,
                                    myClust,
                                    myClassifier, verbose=T,
                                    argForClassif=inputType,
                                    argForModif=inputType,
                                    advMethod=modif,
                                    slot=mySlot)
                        # Export results
                        write.table(max_temp@values, filePath)
                    }
                })
            }
        }
        # CGD attack
        if(myAttack == "CGD"){
            alpha <- as.numeric(aSplit[3])
            epsilon <- as.numeric(aSplit[4])
            if(monoOrMulti == "mono"){
                for (myCluster in unique(clusters)){
                    expID <- paste0(strClassifier,"_cellByCell_alpha=",alpha,"_epsilon=",epsilon, "__",stringr::str_replace_all(myCluster," ","_"),"__")
                    filePath <- paste0(bd,"v2/results/","oneRowSummary_",expID,".txt")
                    message(filePath)
                    # Run only if file does not exist yet
                    if (!file.exists(filePath)){
                        acbc <- advCGD(expr, clusters, myCluster, myClassifier,
                                          genes=mySignGenes$gene[1:500],
                                             exclNewTargets=NULL, newTarget=NULL,
                                             annotMode="max_occ", coef=alpha, add=epsilon,
                                             slot=mySlot, verbose=T)
                        # Export results
                        exportResults(acbc, paste0(bd,"v2/results/"), expID, strClassifier)
                    } else {
                        display(paste(expID, "already done"))
                    }
                }
            } else {
                # Prepare for the multi-threading
                library(parallel)
                cl <- makeCluster(nbThreads)
                all_vars <- ls(globalenv())
                message("bef clustExport")
                clusterExport(cl, varlist = all_vars)
                message("bef par")
                message(paste0(strClassifier,"_",myAttack,"_",modif))
                parLapply(cl, unique(clusters), function(myCluster) {
                    library(adverSCarial)
                    library(scAnnotatR)
                    library(stringr)
                    library(dplyr)
                    library(Seurat)
                    library(HGNChelper)
                    expID <- paste0(strClassifier,"_cellByCell_alpha=",alpha,"_epsilon=",epsilon, "__",stringr::str_replace_all(myCluster," ","_"),"__")
    #                 journalIDs <- read.table(paste0(bd,"v2/results//journal.txt"))$V1
                    filePath <- paste0(bd,"v2/results/","oneRowSummary_",expID,".txt")
                    message(filePath)
                    # Run only if file does not exist yet
                    if (!file.exists(filePath)){
                        acbc <- advCGD(expr, clusters, myCluster, myClassifier,
                                          genes=mySignGenes$gene[1:200],
                                             exclNewTargets=NULL, newTarget=NULL,
                                             annotMode="max_occ", coef=alpha, add=epsilon,
                                             slot=mySlot, verbose=T)
                        # Export results
                        exportResults(acbc, paste0(bd,"v2/results/"), expID, strClassifier)
                    } else {
                        display(paste(expID, "already done"))
                    }
                })
            }
        }
    }
}

# Create lists of attacks to run

In [9]:
# Make a list of all possible combinations
getAttackList <- function(classifiers, alphas, epsilons, attacks, modifs){
    cAttackList <- c()
    for (myClass in classifiers){
        for (attack in attacks){
            if (attack == "cellByCell"){
                for (alpha in alphas){
                   for ( epsilon in epsilons){
                        cAttackList <- c(cAttackList,
                            paste0(myClass,"_",attack,"_",alpha,"_",epsilon))
                   }
                }
            }
            if (attack %in% c("singleGene", "maxChange")){
                for (modif in cModifs){
                    cAttackList <- c(cAttackList,
                            paste0(myClass,"_",attack,"_",modif))
                }
            }
        }
    }
    return(cAttackList)
}

In [10]:
# Make a list of all wanted CGD attacks
getAttackListConcat <- function(classifiers, alphas, epsilons){
    if ( length(alphas) != length(epsilons)){
        stop("alphas and epsilons must have same length")
    }
    cAttackList <- c()
    for (myClass in classifiers){
        for (i in 1:length(alphas)){
            alpha <- alphas[i]
            epsilon <- epsilons[i]
            cAttackList <- c(cAttackList,
                paste0(myClass,"_cellByCell_",alpha,"_",epsilon))                
        }
    }
    return(cAttackList)
}

In [11]:
cClassifiers <- c("CHETAH", "scMLP", "scAnnotatR", "scType", "scRF" )
cAlphas <- c(1)
cEpsilons <- c(1)
cAttacks <- c("maxChange", "singleGene")
cModifs <- c("negativeAberrant", "decile+5","positiveAberrant","perc99", "perc1")

In [12]:
attackList_1 <- getAttackList(cClassifiers, cAlphas, cEpsilons, cAttacks, cModifs)
head(attackList_1)
length(attackList_1)

[1] "CHETAH_maxChange_negativeAberrant"  "CHETAH_maxChange_decile+5"         
[3] "CHETAH_maxChange_positiveAberrant"  "CHETAH_maxChange_perc99"           
[5] "CHETAH_maxChange_perc1"             "CHETAH_singleGene_negativeAberrant"

[1] 50

In [13]:
cAlphas <- c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2)
cEpsilons <- rep(0, length(cAlphas))
cClassifiers <- c("scMLP", "scRF", "scAnnotatR", "scType")
attackList_2 <- getAttackListConcat(cClassifiers, cAlphas, cEpsilons)
head(attackList_2)
length(attackList_2)

[1] "scMLP_cellByCell_0_0"   "scMLP_cellByCell_0.1_0" "scMLP_cellByCell_0.2_0"
[4] "scMLP_cellByCell_0.3_0" "scMLP_cellByCell_0.4_0" "scMLP_cellByCell_0.5_0"

[1] 84

In [14]:
cEpsilons <- c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2)
cAlphas <- rep(0, length(cEpsilons))
cClassifiers <- c("scMLP", "scRF", "scAnnotatR", "scType")
attackList_3 <- getAttackListConcat(cClassifiers, cAlphas, cEpsilons)
head(attackList_3)
length(attackList_3)

[1] "scMLP_cellByCell_0_0"   "scMLP_cellByCell_0_0.1" "scMLP_cellByCell_0_0.2"
[4] "scMLP_cellByCell_0_0.3" "scMLP_cellByCell_0_0.4" "scMLP_cellByCell_0_0.5"

[1] 84

In [16]:
cAlphas <- c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2)
cEpsilons <- c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2)
cClassifiers <- c("scMLP", "scRF", "scAnnotatR", "scType")
attackList_4 <- getAttackListConcat(cClassifiers, cAlphas, cEpsilons)
head(attackList_4)
length(attackList_4)

[1] "scMLP_cellByCell_0_0"     "scMLP_cellByCell_0.1_0.1"
[3] "scMLP_cellByCell_0.2_0.2" "scMLP_cellByCell_0.3_0.3"
[5] "scMLP_cellByCell_0.4_0.4" "scMLP_cellByCell_0.5_0.5"

[1] 84

In [22]:
runAttacks(attackList_1,monoOrMulti="mono")
runAttacks(attackList_2,monoOrMulti="mono")
runAttacks(attackList_3,monoOrMulti="mono")
runAttacks(attackList_4,monoOrMulti="mono")

[1] "MLP"

[1] "data.frame"

data/v2/results/oneRowSummary_MLP_cellByCell_alpha=2_epsilon=0__Memory_CD4_T__.txt



[1] "MLP_cellByCell_alpha=2_epsilon=0__Memory_CD4_T__ already done"

data/v2/results/oneRowSummary_MLP_cellByCell_alpha=2_epsilon=0__B__.txt



...

 mean 0.981088931069655 delt -0.000791772323496254

Number of CD8 T : 4

 mean 0.565582767128944 delt 0

Number of modified cells 68

RPL31 370

Number of original annot NK : 68

 mean 0.980842990033767 delt -0.000245941035887709

Number of CD8 T : 4

 mean 0.565582767128944 delt 0

Number of modified cells 68

CD72 371

Number of original annot NK : 68

 mean 0.980942292248501 delt 9.93022147346423e-05

Number of CD8 T : 4

 mean 0.565582767128944 delt 0

Number of modified cells 68

GIMAP5 372

Number of original annot NK : 68

 mean 0.98098325028139 delt 4.09580328885051e-05

Number of CD8 T : 4

 mean 0.565582767128944 delt 0

Number of modified cells 68

LGALS3 373

Number of original annot NK : 68

 mean 0.981139680918525 delt 0.000156430637135219

Number of CD8 T : 4

 mean 0.565582767128944 delt 0

Number of modified cells 68

RPS4Y1 374

Number of original annot NK : 68

 mean 0.981473660644363 delt 0.000333979725837708

Number of CD8 T : 4

 mean 0.565582767128944 delt 0

Num

 mean 0.98957104380451 delt 2.20083478671418e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

XCL1 416

Number of original annot NK : 67

 mean 0.989309799315324 delt -0.000261244489185852

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

SLC31A2 417

Number of original annot NK : 67

 mean 0.986967155292853 delt -0.00234264402247186

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

RPL15 418

Number of original annot NK : 67

 mean 0.987103306535465 delt 0.000136151242612015

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

FCRLA 419

Number of original annot NK : 67

 mean 0.986611223932522 delt -0.000492082602942201

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

APP 420

Number of original annot NK : 67

 mean 0.986671500241579 delt 6.02763090561398e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number

 mean 0.987730256657102 delt 1.31041256349329e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

RPL10A 462

Number of original annot NK : 67

 mean 0.987770161521969 delt 3.99048648662692e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

CXCR4 463

Number of original annot NK : 67

 mean 0.987835495329615 delt 6.5333807646395e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

RPLP0 464

Number of original annot NK : 67

 mean 0.987923814289605 delt 8.83189599905077e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

CD40 465

Number of original annot NK : 67

 mean 0.987856475274954 delt -6.73390146511688e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Number of modified cells 67

CXCL16 466

Number of original annot NK : 67

 mean 0.987918680283561 delt 6.22050086064307e-05

Number of CD8 T : 5

 mean 0.554464077949524 delt 0

Numbe

[1] "NK"                 "72"                 "CD8 T"             
 [4] "1.6"                "0"                  "0"                 
 [7] "500"                "500"                "0.986111111111111" 
[10] "0.0138888888888889" "0.930555555555556"  "0.0694444444444444"
[13] "0.988241177880102"  "0.0117426033893637" "0.947981845173571" 
[16] "0.0478898756879309"

data/v2/results/oneRowSummary_MLP_cellByCell_alpha=1.6_epsilon=0__CD8_T__.txt

CD8 T

13189

1318

139

New cluster target: NK

HLA.DRA 1

Number of original annot CD8 T : 107

 mean 0.809515919083747 delt 0

Number of NK : 25

 mean 0.815118626356125 delt 0

Number of modified cells 114

PRF1 2

Number of original annot CD8 T : 107

 mean 0.811208702693476 delt 0.00169278360972902

Number of NK : 25

 mean 0.815118626356125 delt 0

Number of modified cells 114

NKG7 3

Number of original annot CD8 T : 105

 mean 0.813984016577403 delt 0.00277531388392693

Number of NK : 27

 mean 0.793437938999247 delt -0.0216806873568782

Number of modified cells 112

FCER1A 4

Number of original annot CD8 T : 103

 mean 0.79615269789418 delt -0.017831318683223

Number of NK : 27

 mean 0.793437938999247 delt 0

Number of modified cells 112

TYROBP 5

Number of original annot CD8 T : 103

 mean 0.79706657540451 delt 0.000913877510330008

Number of NK : 27

 mean 0.793437938999247 delt 0

Number of mo

In [23]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.9.0 
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.9.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=fr_FR.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=fr_FR.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=fr_FR.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=fr_FR.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Paris
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] IRdisplay_1.1      adverSCarial_1.3.6 HGNChelper_0.8.1   Seurat_5.0.3      
[5] SeuratObject_5.0.1 sp_1.6-1           dplyr_1.1.2       

loaded via a namespace (and not attached):
  [1] deldir_1.0-9         